<a href="https://colab.research.google.com/github/OmarMachuca851/Task/blob/main/CNN2D_sprint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN 2D desde scratch

In [ ]:
import numpy as np
from keras.datasets import mnist
from sklearn.model_selection import train_test_split


class ReLU:
    def forward(self, X):
        self.X = X
        return np.maximum(0, self.X)

    def backward(self, dZ):
        return dZ * (self.X > 0).astype(float)


class Softmax:
    def forward(self, X):
        exp_X = np.exp(X - np.max(X, axis=1, keepdims=True))
        return exp_X / np.sum(exp_X, axis=1, keepdims=True)

    def backward(y_pred, y_true):
        batch_size = y_true.shape[0]
        grad = y_pred.copy()
        grad[range(batch_size), y_true] -= 1
        return grad / batch_size


class AdaGrad:
    def __init__(self, lr):
        self.lr = lr
        self.HW = 1
        self.HB = 1
    def update(self, layer):
        self.HW += layer.dW**2
        self.HB += layer.dB**2
        layer.W -= self.lr * np.sqrt(1/self.HW) * layer.dW
        layer.b -= self.lr * np.sqrt(1/self.HB) * layer.dB
        return layer

## Problema 1: Creación de una capa de convolución

In [ ]:
class Conv2D:
    def __init__(self, input_channels, num_filters, kernel_size, lr=0.01, stride=1, padding=0):
        self.input_channels = input_channels
        self.num_filters = num_filters
        self.kh, self.kw = kernel_size
        self.stride = stride
        self.padding = padding
        self.AdaGrad = AdaGrad(lr)

        # Xavier Initializer
        std = np.sqrt(1.0 / (self.kh * self.kw * input_channels))

        self.W = np.random.randn(self.kh, self.kw, input_channels, num_filters) * std
        self.b = np.zeros((1, 1, 1, num_filters))

    # Forward 2D convolution
    def forward(self, X):
        self.X = X
        batch_size, in_height, in_width, in_channels = X.shape

        out_height = (in_height + 2 * self.padding - self.kh + 1) // self.stride
        out_width = (in_width + 2 * self.padding - self.kw + 1) // self.stride

        self.output = np.zeros((batch_size, out_height, out_width, self.num_filters))

        if self.padding > 0:
            X = np.pad(X, ((0, 0), (self.padding, self.padding), (self.padding, self.padding), (0, 0)), mode='constant')

        self.X_padded = X

        # Convolution
        for i in range(out_height):
            for j in range(out_width):
                for k in range(self.num_filters):
                    h_start = i * self.stride
                    h_end = h_start + self.kh
                    w_start = j * self.stride
                    w_end = w_start + self.kw
                    region = X[:, h_start:h_end, w_start:w_end, :]
                    self.output[:, i, j, k] = np.sum(region * self.W[:, :, :, k], axis=(1, 2, 3)) + self.b[0, 0, 0, k]

        return self.output

    # Backward 2D convolution
    def backward(self, dZ):
        _, out_height, out_width, _ = dZ.shape
        dX = np.zeros_like(self.X_padded, dtype=np.float32)
        self.dW = np.zeros_like(self.W)
        self.dB = np.zeros_like(self.b)

        # Gradients
        for i in range(out_height):
            for j in range(out_width):
                for k in range(self.num_filters):
                    h_start = i * self.stride
                    h_end = h_start + self.kh
                    w_start = j * self.stride
                    w_end = w_start + self.kw
                    region = self.X_padded[:, h_start:h_end, w_start:w_end, :]
                    self.dW[:, :, :, k] += np.sum(region * dZ[:, i, j, k][:, None, None, None], axis=0)
                    self.dB[0, 0, 0, k] += np.sum(dZ[:, i, j, k])
                    dX[:, h_start:h_end, w_start:w_end, :] += self.W[:, :, :, k] * dZ[:, i, j, k][:, None, None, None]

        if self.padding > 0:
            dX = dX[:, :, self.padding : -self.padding, self.padding : -self.padding]

        self.AdaGrad.update(self)

        return dX

## Problema 2: Experimento con capas convolucionales 2D en matrices pequeñas

In [ ]:
# Input data when flowing CNN2 forwards (1, 4, 4, 1)
x = np.array([[[[1],[ 2], [3], [4]],
                [[5], [6], [7], [8]],
                [[9], [10], [11], [12]],
                [[13], [14], [15], [16]]]])

# Manually setting filters
w = np.array([
    [[[0, 0]], [[0, 0]], [[0, 0]]],
     [[[0, 0]], [[1, -1]], [[0, 1]]],
     [[[0, 0]], [[-1, 0]], [[0, 0]]]
]).astype(np.float32) # Cast to float

b = np.array([[[[0, 0]]]], dtype=np.float32)

# Conv2 with 1 input channel , 2 output channels, kernel 3x3
conv = Conv2D(input_channels=1, num_filters=2, kernel_size=(3, 3), lr=0.01, stride=1, padding=0)
conv.W = w.copy()
conv.B = b.copy()

# Forward pass
out = conv.forward(x)
print(f"forward Output:\n{out}")

# Backward test
dout = np.array([[[[-4, 1], [-4, -7]], [[-4, 1], [-4, -11]]]], dtype=np.float32)
dx = conv.backward(dout)
print(f'\nBackward Output (dx): \n{dx.reshape(1, 1, 4, 4)}')
print('\n')

forward Output:
[[[[-4.  1.]
   [-4.  1.]]

  [[-4.  1.]
   [-4.  1.]]]]

Backward Output (dx): 
[[[[  0.   0.   0.   0.]
   [  0.  -5.   4.  -7.]
   [  0.  -1.  12. -11.]
   [  0.   4.   4.   0.]]]]




## Problema 3: Tamaño de salida después de la convolución 2D

In [ ]:
def conv2d_output_size(H_in, W_in, kernel_size, stride=1, padding=0):
    kh, kw = kernel_size
    H_out = (H_in + 2 * padding - kh) // stride + 1
    W_out = (W_in + 2 * padding - kw) // stride + 1
    return H_out, W_out

## Problema 4: Creación de una capa de agrupación máxima

In [ ]:
class MaxPool2D:
    def __init__(self, pool_size=(2, 2), stride=2):
        self.ph, self.pw = pool_size
        self.stride = stride

    def forward(self, X):
        self.X = X
        batch_size, in_height, in_width, in_channels = X.shape

        out_height = (in_height - self.ph) // self.stride + 1
        out_width = (in_width - self.pw) // self.stride + 1

        self.output = np.zeros((batch_size, out_height, out_width, in_channels))
        self.max_indices = np.zeros((batch_size, out_height, out_width, in_channels, 2), dtype=int)

        for i in range(out_height):
            for j in range(out_width):
                for c in range(in_channels):
                    h_start = i * self.stride
                    h_end = h_start + self.ph
                    w_start = j * self.stride
                    w_end = w_start + self.pw

                    region = X[:, h_start:h_end, w_start:w_end, c]
                    self.output[:, i, j, c] = np.max(region, axis=(1, 2))

                    max_indices_flat = np.argmax(region.reshape(batch_size, -1), axis=1)
                    for b in range(batch_size):
                        h_idx = max_indices_flat[b] // self.ph
                        w_idx = max_indices_flat[b] % self.pw
                        self.max_indices[b, i, j, c] = [h_start + h_idx, w_start + w_idx]

        return self.output

    def backward(self, dZ):
        batch_size, out_height, out_width, out_channels = dZ.shape
        dX = np.zeros_like(self.X)

        for i in range(out_height):
            for j in range(out_width):
                for c in range(out_channels):
                    for b in range(batch_size):
                        h_idx, w_idx = self.max_indices[b, i, j, c]
                        dX[b, h_idx, w_idx, c] += dZ[b, i, j, c]

        return dX

## Problema 5: Creación de agrupamiento promedio

In [ ]:
class AveragePool2D:
    def __init__(self, pool_size=(2, 2), stride=2):
        self.ph, self.pw = pool_size
        self.stride = stride

    def forward(self, x):
        self.x = x
        N, C, H, W = x.shape
        out_h = (H - self.ph) // self.stride + 1
        out_w = (W - self.pw) // self.stride + 1
        self.arg_max = np.zeros((N, C, out_h, out_w), dtype=np.int32)

        out = np.zeros((N, C, out_h, out_w))

        for n in range(N):
            for c in range(C):
                for i in range(out_h):
                    for j in range(out_w):
                        h_start = i * self.stride
                        w_start = j * self.stride
                        window = x[n, c, h_start : h_start + self.ph, w_start : w_start + self.pw]
                        out[n, c, i, j] = np.mean(window)
        return out

    def backward(self, dout):
        N, C, H, W = self.x.shape
        out_h, out_w = dout.shape[2:]
        dx = np.zeros_like(self.x)

        for n in range(N):
            for c in range(C):
                for i in range(out_h):
                    for j in range(out_w):
                        h_start = i * self.stride
                        w_start = j * self.stride
                        dx[n, c, h_start : h_start + self.ph, w_start : w_start + self.pw] += dout[n, c, i, j] / (self.ph  + self.pw)
        return dx

## Problema 6: Suavizado

In [ ]:
class Flatten:
    def __init__(self):
        self.input_shape = None

    def forward(self, X):
        self.input_shape = X.shape
        return X.reshape(X.shape[0], -1)

    def backward(self, dZ):
        return dZ.reshape(self.input_shape)

## Problema 7: Aprendizaje y estimación

In [ ]:
class Scratch2dCNNClassifier:
    def __init__(self, CNN, epochs=10, batch_size=20, verbose=True):
        self.layers = CNN
        self.epochs = epochs
        self.batch_size = batch_size
        self.verbose = verbose

    def forward(self, X):
        self.activations = [X]

        for layer in self.layers:
            output = layer.forward((self.activations[-1]))
            self.activations.append(output)

        return self.activations[-1]

    def backward(self, y_pred, y_true):
        grad = Softmax.backward(y_pred, y_true)

        for i in range(len(self.layers) - 2, -1, -1):
            layer = self.layers[i]
            grad = layer.backward(grad)

    def cross_entropy_loss(self, y_pred, y_true):
        m = y_true.shape[0]
        log_likelihood = -np.log(y_pred[range(m), y_true])
        return np.sum(log_likelihood) / m

    def accuracy(self, y_pred, y_true):
        predictions = np.argmax(y_pred, axis=1)
        return np.mean(predictions == y_true)

    def fit(self, X, y, validation_data=None, verbose=True):
        n_samples = X.shape[0]

        for epoch in range(self.epochs):
            epoch_loss = 0
            indices = np.random.permutation(n_samples)
            X_shuffled = X[indices]
            y_shuffled = y[indices]

            for i in range(0, n_samples, self.batch_size):
                X_batch = X_shuffled[i:i+self.batch_size]
                y_batch = y_shuffled[i:i+self.batch_size]

                # Forward pass
                y_pred = self.forward(X_batch)

                loss = self.cross_entropy_loss(y_pred, y_batch)
                epoch_loss += loss

                # Backward pass
                self.backward(y_pred, y_batch)

            avg_loss = epoch_loss / (n_samples / self.batch_size)

            # Validation
            if validation_data is not None:
                X_val, y_val = validation_data
                val_pred = self.forward(X_val)
                val_loss = self.cross_entropy_loss(val_pred, y_val)
                val_acc = self.accuracy(val_pred, y_val)

            if verbose:
                if validation_data is not None:
                    print(f"Epoch {epoch+1:3d}/{self.epochs} | "
                          f"Train Loss: {avg_loss:.4f} | "
                          f"Val Loss: {val_loss:.4f} | "
                          f"Val Acc: {val_acc:.4f}")
                else:
                    print(f"Epoch {epoch+1:3d}/{self.epochs} | Train Loss: {avg_loss:.4f}")

    def predict(self, X):
        y_pred = self.forward(X)
        return np.argmax(y_pred, axis=1)

    def predict_proba(self, X):
        return self.forward(X)

In [ ]:
class FC:
    def __init__(self, input_size, output_size, lr=0.01):
        self.input_size = input_size
        self.output_size = output_size
        self.lr = lr
        self.AdaGrad = AdaGrad(lr)

        # Xavier initializer
        std = np.sqrt(1.0 / input_size)

        self.W = np.random.randn(input_size, output_size) * std
        self.b = np.zeros((1, output_size))

    def forward(self, X):
        self.X = X
        return np.dot(X, self.W) + self.b

    def backward(self, dZ):
        self.dW = np.dot(self.X.T, dZ)
        self.dB = np.sum(dZ, axis=0, keepdims=True)
        dX = np.dot(dZ, self.W.T)
        self.AdaGrad.update(self)
        return dX

In [ ]:
def load_and_preprocess_data():
    # Loading MNIST data
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    X_train, y_train, X_test, y_test = X_train[:10000], y_train[:10000], X_test[:2000], y_test[:2000]
    # Preprocessimg
    X_train = X_train.reshape(-1, 28, 28, 1).astype(np.float32) / 255
    X_test = X_test.reshape(-1, 28, 28, 1).astype(np.float32) / 255

    # Splitting into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

    return X_train, y_train, X_val, y_val, X_test, y_test

X_train, y_train, X_val, y_val, X_test, y_test = load_and_preprocess_data()

In [ ]:
CN = [
    Conv2D(1, 8, kernel_size=(3, 3), lr=0.01, stride=1, padding=1),
    ReLU(),
    MaxPool2D(pool_size=(2, 2), stride=2),
    Flatten(),
    FC(14*14*8, 64),
    ReLU(),
    FC(64, 10),
    Softmax(),
]
cnn = Scratch2dCNNClassifier(CN, epochs=5, batch_size=50)

cnn.fit(X_train, y_train, validation_data=(X_val, y_val))

y_train_pred = cnn.predict_proba(X_train)
y_test_pred = cnn.predict_proba(X_test)

train_accuracy = cnn.accuracy(y_train_pred, y_train)
test_accuracy = cnn.accuracy(y_test_pred, y_test)

print(f"{train_accuracy = :.4f}")
print(f"{test_accuracy = :.4f}")

Epoch   1/5 | Train Loss: 1.6477 | Val Loss: 0.9615 | Val Acc: 0.8085
Epoch   2/5 | Train Loss: 0.7062 | Val Loss: 0.5265 | Val Acc: 0.8715
Epoch   3/5 | Train Loss: 0.4794 | Val Loss: 0.4245 | Val Acc: 0.8855
Epoch   4/5 | Train Loss: 0.4058 | Val Loss: 0.3759 | Val Acc: 0.9040
Epoch   5/5 | Train Loss: 0.3653 | Val Loss: 0.3489 | Val Acc: 0.9035
train_accuracy = 0.9049
test_accuracy = 0.8660


## Problema 8: LeNet

In [ ]:
LeNet = [
    Conv2D(1, 6, kernel_size=(5, 5), lr=0.01, stride=1, padding=0),
    ReLU(),
    MaxPool2D(pool_size=(2, 2), stride=2),
    Conv2D(6, 16, kernel_size=(5, 5), lr=0.01, stride=1, padding=0),
    ReLU(),
    MaxPool2D(pool_size=(2, 2), stride=2),
    Flatten(),
    FC(16*4*4, 200),
    ReLU(),
    FC(200, 120),
    ReLU(),
    FC(120, 10),
    Softmax(),
]
cnn = Scratch2dCNNClassifier(CN, epochs=5, batch_size=50)

cnn.fit(X_train, y_train,validation_data=(X_val, y_val))

y_train_pred = cnn.predict_proba(X_train)
y_test_pred = cnn.predict_proba(X_test)

train_accuracy = cnn.accuracy(y_train_pred, y_train)
test_accuracy = cnn.accuracy(y_test_pred, y_test)

print(f"{train_accuracy = :.4f}")
print(f"{test_accuracy = :.4f}")

Epoch   1/5 | Train Loss: 0.3423 | Val Loss: 0.3330 | Val Acc: 0.9085
Epoch   2/5 | Train Loss: 0.3235 | Val Loss: 0.3153 | Val Acc: 0.9140
Epoch   3/5 | Train Loss: 0.3094 | Val Loss: 0.3117 | Val Acc: 0.9145
Epoch   4/5 | Train Loss: 0.2977 | Val Loss: 0.2965 | Val Acc: 0.9190
Epoch   5/5 | Train Loss: 0.2860 | Val Loss: 0.2900 | Val Acc: 0.9185
train_accuracy = 0.9230
test_accuracy = 0.8820


## Problema 9: Investigación sobre modelos famosos de reconocimiento de imágenes

- **LeNet-5 (1998):**
LeNet-5, una red convolucional pionera de 7 niveles, creada por LeCun et al. en 1998 y que clasifica dígitos, fue aplicada por varios bancos para reconocer números manuscritos en cheques digitalizados en imágenes de entrada en escala de grises de 32x32 píxeles. La capacidad de procesar imágenes de mayor resolución requiere capas más grandes y convolucionales, por lo que esta técnica está limitada por la disponibilidad de recursos informáticos.

- **AlexNet (2012):**
En 2012, AlexNet superó significativamente a todos sus competidores anteriores y ganó el desafío al reducir el error del top 5 del 26 % al 15,3 %. La tasa de error del top 5, que obtuvo el segundo puesto y que no fue una variación de CNN, fue de alrededor del 26,2 %.
La red tenía una arquitectura muy similar a la de LeNet de Yann LeCun et al., pero era más profunda, con más filtros por capa y capas convolucionales apiladas. Consistía en convoluciones de 11x11, 5x5 y 3x3, agrupamiento máximo, abandono, aumento de datos, activaciones ReLU y SGD con momentum. Incorporaba activaciones ReLU después de cada capa convolucional y completamente conectada. AlexNet se entrenó durante 6 días simultáneamente en dos GPU Nvidia GeForce GTX 580, razón por la cual su red se divide en dos pipelines. AlexNet fue diseñada por el grupo SuperVision, compuesto por Alex Krizhevsky, Geoffrey Hinton e Ilya Sutskever.

- **ZFNet (2013):**
Como era de esperar, el ganador del ILSVRC 2013 también fue una CNN conocida como ZFNet. Logró una tasa de error del 14,8 %, que se encuentra entre las 5 mejores, lo que representa la mitad de la tasa de error no neuronal mencionada anteriormente. Este logro se logró principalmente mediante la optimización de los hiperparámetros de AlexNet, manteniendo la misma estructura con elementos adicionales de aprendizaje profundo, como se mencionó anteriormente en este ensayo.

- **GoogleNet/Origen (2014):**
El ganador de la competición ILSVRC 2014 fue GoogLeNet (también conocido como Inception V1) de Google. ¡Logró una tasa de error del 6,67%, entre las 5 mejores! Esto se acercaba mucho al rendimiento humano, que los organizadores del desafío ahora debían evaluar. Resultó ser bastante difícil y requirió entrenamiento humano para superar la precisión de GoogLeNet. Tras unos días de entrenamiento, el experto (Andrej Karpathy) logró una tasa de error del 5,1% (modelo único) y del 3,6% (modelo conjunto), entre las 5 mejores. La red utilizó una CNN inspirada en LeNet, pero implementó un elemento novedoso denominado módulo Inception. Utilizaba normalización por lotes, distorsiones de imagen y RMSprop. Este módulo se basa en varias convoluciones muy pequeñas para reducir drásticamente el número de parámetros. Su arquitectura consistía en una CNN de 22 capas, pero redujo el número de parámetros de 60 millones (AlexNet) a 4 millones.

- **VGGNet (2014):**
El subcampeón de la competencia ILSVRC 2014, VGGNet, fue desarrollado por Simonyan y Zisserman y bautizado por la comunidad como VGGNet. VGGNet consta de 16 capas convolucionales y es muy atractivo gracias a su arquitectura uniforme. Similar a AlexNet, solo convoluciones 3x3, pero con numerosos filtros. Se entrenó en 4 GPU durante 2-3 semanas. Actualmente, es la opción preferida de la comunidad para extraer características de imágenes. La configuración de pesos de VGGNet está disponible públicamente y se ha utilizado en muchas otras aplicaciones y desafíos como extractor de características de referencia. Sin embargo, VGGNet consta de 138 millones de parámetros, lo que puede resultar un poco complejo de manejar.

- **ResNet (2015):**
Finalmente, en el ILSVRC 2015, la denominada Red Neuronal Residual (ResNet) de Kaiming He et al. introdujo una novedosa arquitectura con conexiones de salto y una alta normalización por lotes. Estas conexiones de salto, también conocidas como unidades con compuerta o unidades recurrentes con compuerta, presentan una gran similitud con elementos recientes y exitosos aplicados en las redes neuronales recurrentes (RNN). Gracias a esta técnica, lograron entrenar una red neuronal con 152 capas, manteniendo una complejidad menor que la de VGGNet. Alcanza una tasa de error del 3,57 %, dentro de los 5 primeros, lo que supera el rendimiento humano en este conjunto de datos.

AlexNet tiene dos líneas CNN paralelas de entrenadas en dos GPU con conexiones cruzadas, GoogleNet tiene módulos de inicio y ResNet tiene conexiones residuales.

## Problema 10: Cálculo del tamaño de salida y el número de parámetros

In [ ]:
def compute_conv_output_and_params(H_in, W_in, C_in, kernel_size, C_out, stride=1, padding=0):
    kh, kw = kernel_size

    # Output dimensions
    H_out = (H_in + 2 * padding - kh) // stride + 1
    W_out = (W_in + 2 * padding - kw) // stride + 1

    # Parameters per filter: C_in * kh * Kw, plus 1 bias output channel
    params_per_filter = C_in * kh * kw + 1
    total_params = params_per_filter * C_out

    return (H_out, W_out, C_out), total_params

# 1. Input: 144x144x3, filter 3x3, 6 filters, stride=1, pading=0
out_size1, params1 = compute_conv_output_and_params(144, 144, 3, (3, 3), 6)
print(f"Layer 1 Output:{out_size1} Params:{params1}")

# 1. Input: 60x60x24, filter 3x3, 48 filters, stride=1, pading=0
out_size2, params2 = compute_conv_output_and_params(60, 60, 24, (3, 3), 48)
print(f"Layer 1 Output:{out_size2} Params:{params2}")

# 1. Input: 20x20x10, filter 3x3, 20 filters, stride=, pading=0
out_size3, params3 = compute_conv_output_and_params(20, 20, 10, (3, 3), 20, stride=2)
print(f"Layer 1 Output:{out_size3} Params:{params3}")

Layer 1 Output:(142, 142, 6) Params:168
Layer 1 Output:(58, 58, 48) Params:10416
Layer 1 Output:(9, 9, 20) Params:1820


## Problema 11: sobre el tamaño de filtors

- **¿Por qué los filtros 3x3 se utilizan comunmente en lugar de los mas gerandes como 7x7?**

El uso de los filtros convolucionales de menor tamaño,por ejemplo 3x3 en lugar de grandes filtros convolucionales, por ejemplo 7x7, es convención común de las recientes arquitecturas de CNN dedido a multiples razones, primero el apilamiento residual con varias capas compuestas de filtros 3x3 es más no lineal para la red. Las funciones de activación ocurren despues de cada capa, por lo tanto, con tres capas 3x3 se pueden realizar mas operaciones de activación que con una capa 7x7, y de esta manera son posibles redes mas profundas y expresivas. En segundo lugar los filtros 3x3 tienen un mejor uso de parámetros, por ejemplo en esta capa de 7x7 con canales de entrada y salida, se tendría 49C 2 parámetros, pero con 3 capas convolucionales 3x3 apilados uno encima de otro solo necesitariamos 27C 2 parámetros. Las 3 capas 3x3 puede abarcar el mismo campo de influencia que dos capas 7x7 a pesar del número de parametros reducidos, La red podría comprender la misma información espacial con el beneficio de marginal mente más capacidades de aprendizaje y menos cálculos.

- **efecto de un filtor 1x1 sin altura ni ancho:**

un filtro de convcolución 1x1 sin altura ni ancho no puede extraer información espacial. Mas bien, actua sobre la dimensión del canal y tiene el efecto de realizar instantaneamente una capa totalmente conectada en cada posición de píxeles en la imagen. Las aplicaciones primarias de un filtor 1x1 son: reducción de la dimensionalidad de entrada, Reducción de lnúmero de canales antes de someterse a una convolución mas compleja en terminos de cálculos e integrar la red con profundidad adicional sin expandirla espacialmente. Además, el aprendisaje de la convinación no lineal en los canales de caracteristicas se pueden hacer usando filtros 1x1. La idea fue concebida  en la arquitectura de la red (NIN) y ha sido implementado con éxito por GoogleNet, en el que fue fundamental para reducir la pronfundidad de las redes y mejorar la eficiente.